In [1]:
import numpy as np
import cv2
import time as t
import matplotlib.pyplot as plt
import math

In [2]:
def no(x):
    a = 0

In [3]:
def read_trackbars():
    hl = cv2.getTrackbarPos('h', 'low')
    sl = cv2.getTrackbarPos('s', 'low')
    vl = cv2.getTrackbarPos('v', 'low')
    
    hh = cv2.getTrackbarPos('h', 'high')
    sh = cv2.getTrackbarPos('s', 'high')
    vh = cv2.getTrackbarPos('v', 'high')

In [4]:
def setup_trackbars():
    cv2.namedWindow('low')
    cv2.namedWindow('high')
    cv2.createTrackbar('h', 'low', 0, 179, no)
    cv2.createTrackbar('s', 'low', 0, 255, no)
    cv2.createTrackbar('v', 'low', 0, 255, no)
    cv2.createTrackbar('h', 'high', 0, 179, no)
    cv2.createTrackbar('s', 'high', 0, 255, no)
    cv2.createTrackbar('v', 'high', 0, 255, no)
    cv2.createTrackbar('ratio_low', 'low', 100, 300, no)
    cv2.createTrackbar('ratio_high', 'high', 100, 300, no)
    cv2.createTrackbar('epsilon', 'low', 0, 100, no)
    cv2.createTrackbar('pp_low', 'low', 0, 1000, no)
    cv2.createTrackbar('pp_high', 'high', 0, 1000, no)

In [5]:
def draw_cross(img, x, y, length, color):
    img = cv2.line(img, (x, y), (x, y + length), color ,5)
    img = cv2.line(img, (x, y), (x, y - length), color ,5)
    img = cv2.line(img, (x, y), (x + length, y), color ,5)
    img = cv2.line(img, (x, y), (x - length, y), color ,5)
    return img

In [7]:
cap.release()
cv2.destroyAllWindows()

In [6]:
cap = cv2.VideoCapture(0)
setup_trackbars()
hl = sl = vl = hh = sh = vh = 0
hight = [hh, sh, vh]
lowt = [hl, sl, vl]

lowi = np.zeros((200, 200, 3), np.uint8)
highi = np.zeros((200, 200, 3), np.uint8)
time2 = t.time()

while True:
    __, img = cap.read()
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hl = cv2.getTrackbarPos('h', 'low')
    sl = cv2.getTrackbarPos('s', 'low')
    vl = cv2.getTrackbarPos('v', 'low')
    
    hh = cv2.getTrackbarPos('h', 'high')
    sh = cv2.getTrackbarPos('s', 'high')
    vh = cv2.getTrackbarPos('v', 'high')
    
    low_ratio = cv2.getTrackbarPos('ratio_low', 'low')/100
    high_ratio = cv2.getTrackbarPos('ratio_high', 'high')/100
    e = cv2.getTrackbarPos('epsilon', 'low')/100
    low_pp = cv2.getTrackbarPos('pp_low', 'low')
    high_pp = cv2.getTrackbarPos('pp_high', 'high')
    
    hight = np.array([hh, sh, vh], np.uint8)
    highi[:] = [hh, sh, vh]
    highc = cv2.cvtColor(highi, cv2.COLOR_HSV2BGR)
    
    lowt = np.array([hl, sl, vl], np.uint8)
    lowi[:] = [hl, sl, vl]
    lowc = cv2.cvtColor(lowi, cv2.COLOR_HSV2BGR)
    
    
    
    mask = cv2.inRange(hsv, lowt, hight)
    kernel = np.ones((5, 5), np.uint8)/25
    proc = cv2.erode(mask, kernel, 1)
    kernel = np.ones((10, 10), np.uint8)/100
    proc = cv2.dilate(proc, kernel, 1)
    mask = proc
    
    con, hier = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if len(con) > 0:
        max_size = 0
        i_max_size = 0
        for i in range (len(con)):
            rect = cv2.minAreaRect(con[i])
            area = rect[1][0]*rect[1][1]
            ratio = rect[1][1]/(rect[1][0]+0.01)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            if ratio < 1:
                ratio = 1/(ratio+0.01)
            img = cv2.drawContours(img,[box],0,(0,0,255),2)
            pp = cv2.arcLength(con[i],True)**2/(cv2.contourArea(con[i])+0.0001)
            #print(low_pp, pp, high_pp)
            if max_size < area:
                #print(1)
                if ratio > low_ratio and ratio < high_ratio:
                    #print(2)
                    if pp > low_pp and pp < high_pp:
                        #print(3)
                        max_size = area
                        i_max_size = i
        if max_size > 10:
            img = cv2.drawContours(img, con[i_max_size], -1, (0,255,0), 1)
            cnt = con[i_max_size]
            M = cv2.moments(cnt)
            cx = int(M['m10']/(M['m00']+0.001))
            cy = int(M['m01']/(M['m00']+0.001))
            img = draw_cross(img, cx, cy, 20, (0, 255, 0))
            clist = list_crt(con[i_max_size], 1)
            img = cv2.drawContours(img, clist, -1, (255,0,0), 8)
            epsilon = e*cv2.arcLength(cnt,True)
            approx = cv2.approxPolyDP(cnt,epsilon,True)
            a2 = list_crt(approx, 1)
            img = cv2.drawContours(img, a2, -1, (0,0,255), 20)
            a3 = list_crt(approx, 2)
            img = cv2.drawContours(img, a3, -1, (0,255,0), 15)
            img = cv2.drawContours(img, approx, -1, (255,0,0), 8)
            if t.time() -time2 > 3:
                time2 = t.time()
                rect = cv2.minAreaRect(con[i_max_size])
                ratio = rect[1][1]/(rect[1][0]+0.01)
                if ratio < 1:
                    ratio = 1/(ratio+0.01)
                print("ratio = ", ratio)
                print(cv2.arcLength(cnt,True)**2/cv2.contourArea(cnt), cv2.isContourConvex(cnt))
                print(n_turns(approx)/math.pi)
                print("  ")
    
    
    
    cv2.imshow('low', lowc)
    cv2.imshow('high', highc)
    cv2.imshow('img', img)
    cv2.imshow('mask', mask)
    cv2.imshow('proc', proc)
    
    if cv2.waitKey(1) != 255:
        break

cap.release()
cv2.destroyAllWindows()

In [23]:
def n_turns(cont):
    c = len(cont)
    angl = 0
    n = 0
    for i in range (c):
        k1, l1 = line(cont[i], cont[(i+1)%c])
        k2, l2 = line(cont[(i+1)%c], cont[(i+2)%c])
        k3, l3 = line(cont[i], cont[(i+2)%c])
        anglp, temp = find_angle(k1, k2, k3, l3, cont[i], cont[(i+1)%c], cont[(i+2)%c])
        n+= temp
        angl += anglp
    #print("n=", n)
    return angl
        

In [24]:
def line(a, b):
    a = a[0]
    b = b[0]
    #print("a=", a, " b=", b)
    if (b[0]-a[0]) != 0:
        k = (b[1]-a[1])/(b[0]-a[0])
        l = a[1] - k*a[0]
        if k > 100000:
            return k, a[0]
        return k, l
    else:
        return 100000, a[0] 

In [35]:
def find_angle(k1, k2, k3, l3, a, b, c):
    a = a[0]
    b = b[0]
    c = c[0]
    anglr = 0
    n = 0
    if (1+k1*k2) != 0:
        anglr = math.atan(abs((k1-k2)/(1+k1*k2)))
    else:
        anglr = math.pi / 2
    
    if a[0] < c[0]:
        if k1 < 100000:
            if b[1] < k3*b[0]+l3:
                anglr = 2*math.pi - anglr
                n+= 1
        else:
            if b[0] > l3:
                anglr = 2*math.pi - anglr
                n+= 1
                
    else:
        if k1 < 100000:
            if b[1] > k3*b[0]+l3:
                anglr = 2*math.pi - anglr
                n+= 1
        else:
            if b[0] < l3:
                anglr = 2*math.pi - anglr
                n+= 1
    print("anglr=", math.degrees(anglr))#, " y=", k1, k2, k3, l3, a, b, c)
    return anglr, n
    

In [26]:
def list_crt(c, n):
    ret = []
    for i in range (n):
        ret.append(c[i])
    return ret